# Agno BiasafeAI Portfolio Optimization Playground - Custom Asset Selection

This notebook provides an interactive environment for testing the agno_biasafe portfolio optimization functionality with **CUSTOM ASSET SELECTION**.

## 🎯 New Features: Custom Asset Input
All workflows now support custom asset selection:
- **Portfolio Optimization**: Choose your own assets for optimization
- **Risk Analysis**: Analyze risk for your specific portfolio  
- **Data Analysis**: Market analysis for your chosen assets
- **Backtesting**: Test strategies on your asset universe

## 🚀 Quick Examples
```python
# Tech Portfolio
assets = "AAPL,MSFT,GOOGL,NVDA,TSLA"

# Sector ETFs  
assets = "XLK,XLF,XLE,XLV,XLI"

# International Diversification
assets = "VTI,VXUS,VWO,VEA,IEMG"
```

## Setup

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from datetime import datetime, timedelta

# Add current directory to path for imports
current_dir = os.path.dirname(os.path.abspath('.'))
sys.path.insert(0, current_dir)

# Import agno client
from main import create_client

print("✅ Libraries imported successfully")

## 1. Initialize Client and Test Connection

In [ ]:
# Create the agno client
client = create_client()

# Test API connection
try:
    health = client.health_check()
    print(f"✅ API Connection Successful: {health}")
except Exception as e:
    print(f"❌ API Connection Failed: {e}")
    print("Make sure the riskfolio engine is running on port 5000")

## 2. Generate Sample Data

Let's create some realistic market data for testing.

In [ ]:
# Set random seed for reproducible results
np.random.seed(42)

# Define parameters
n_days = 252  # One year of trading days
assets = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TSLA', 'META']
n_assets = len(assets)

# Generate realistic returns with correlations
mean_returns = np.array([0.0008, 0.0006, 0.0010, 0.0007, 0.0015, 0.0009])  # Daily mean returns
volatilities = np.array([0.025, 0.022, 0.028, 0.024, 0.035, 0.030])  # Daily volatilities

# Create correlation matrix
correlations = np.array([
    [1.00, 0.65, 0.55, 0.70, 0.45, 0.60],
    [0.65, 1.00, 0.60, 0.75, 0.40, 0.65],
    [0.55, 0.60, 1.00, 0.50, 0.35, 0.55],
    [0.70, 0.75, 0.50, 1.00, 0.45, 0.70],
    [0.45, 0.40, 0.35, 0.45, 1.00, 0.50],
    [0.60, 0.65, 0.55, 0.70, 0.50, 1.00]
])

# Convert to covariance matrix
covariance_matrix = np.outer(volatilities, volatilities) * correlations

# Generate returns
returns_data = np.random.multivariate_normal(
    mean=mean_returns,
    cov=covariance_matrix,
    size=n_days
)

# Create DataFrame with dates
end_date = datetime.now()
start_date = end_date - timedelta(days=int(n_days * 1.4))  # Account for weekends
dates = pd.date_range(start=start_date, end=end_date, freq='B')[:n_days]

returns_df = pd.DataFrame(returns_data, index=dates, columns=assets)

print(f"✅ Generated {n_days} days of returns for {n_assets} assets")
print(f"Date range: {dates[0].strftime('%Y-%m-%d')} to {dates[-1].strftime('%Y-%m-%d')}")

# Display basic statistics
returns_df.describe()

## 3. Visualize the Data

In [ ]:
# Plot cumulative returns
plt.figure(figsize=(12, 8))

# Calculate cumulative returns
cumulative_returns = (1 + returns_df).cumprod()

plt.subplot(2, 2, 1)
cumulative_returns.plot(title='Cumulative Returns')
plt.ylabel('Cumulative Return')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot daily returns distribution
plt.subplot(2, 2, 2)
returns_df.boxplot(ax=plt.gca())
plt.title('Daily Returns Distribution')
plt.xticks(rotation=45)

# Plot correlation heatmap
plt.subplot(2, 2, 3)
correlation_matrix = returns_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Returns Correlation Matrix')

# Plot rolling volatility
plt.subplot(2, 2, 4)
rolling_vol = returns_df.rolling(window=30).std() * np.sqrt(252)
rolling_vol.plot(title='30-Day Rolling Volatility (Annualized)')
plt.ylabel('Volatility')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

print("\n📊 Data visualization complete")

## 4. Portfolio Optimization

Now let's optimize portfolios using different objectives and risk measures.

In [ ]:
# Define optimization scenarios with CUSTOM ASSETS
custom_portfolios = {
    "Tech Portfolio": {"assets": "AAPL,MSFT,GOOGL,NVDA,TSLA", "weights": None},
    "Value Portfolio": {"assets": "BRK-B,JPM,JNJ,PG,KO", "weights": None},
    "Sector ETFs": {"assets": "XLK,XLF,XLE,XLV,XLI", "weights": None},
    "International": {"assets": "VTI,VXUS,VWO,VEA,IEMG", "weights": None},
    "Crypto Portfolio": {"assets": "BTC-USD,ETH-USD,ADA-USD", "weights": None}
}

# Test with different portfolios
for portfolio_name, config in custom_portfolios.items():
    print(f"\n🎯 Testing Portfolio: {portfolio_name}")
    print(f"   Assets: {config['assets']}")
    
    # Convert to asset list
    asset_list = [asset.strip().upper() for asset in config['assets'].split(",")]
    
    # Generate sample data for these assets
    returns_data = np.random.multivariate_normal(
        mean=np.random.uniform(0.0005, 0.0015, len(asset_list)),
        cov=np.eye(len(asset_list)) * 0.0004 + np.full((len(asset_list), len(asset_list)), 0.0001),
        size=n_days
    )
    
    returns_df = pd.DataFrame(returns_data, index=dates, columns=asset_list)
    
    # Try optimization with these assets
    try:
        weights = client.portfolio.optimize_portfolio(
            returns=returns_df,
            model="Classic",
            rm="MV",
            obj="MinRisk",
            rf=0.02/252
        )
        
        print(f"   ✅ Optimization successful! Top holdings:")
        sorted_weights = sorted(weights.items(), key=lambda x: x[1], reverse=True)
        for asset, weight in sorted_weights[:3]:
            print(f"      {asset}: {weight:.3f}")
        
    except Exception as e:
        print(f"   ❌ Failed: {e}")

print("\n📈 Custom asset optimization testing complete!")

## 5. Compare Portfolio Weights

In [ ]:
# Create comparison DataFrame
weights_df = pd.DataFrame(optimization_results).fillna(0)

# Remove failed optimizations
weights_df = weights_df.loc[:, weights_df.sum() > 0]

print("Portfolio Weights Comparison:")
print("="*50)
print(weights_df.round(4))

# Plot weights comparison
if len(weights_df.columns) > 0:
    plt.figure(figsize=(12, 8))
    
    # Stacked bar chart
    plt.subplot(2, 1, 1)
    weights_df.T.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title('Portfolio Weights by Strategy')
    plt.ylabel('Weight')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    
    # Individual asset allocation
    plt.subplot(2, 1, 2)
    weights_df.plot(kind='bar', ax=plt.gca())
    plt.title('Asset Allocation by Strategy')
    plt.ylabel('Weight')
    plt.xlabel('Assets')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No successful optimizations to display")

## 6. Risk Metrics Analysis

In [ ]:
# Calculate risk metrics for each successful portfolio
risk_metrics_results = {}

for strategy_name, weights in optimization_results.items():
    if weights is not None:
        print(f"\n📊 Calculating risk metrics for: {strategy_name}")
        
        try:
            # Calculate risk metrics
            metrics = client.risk.calculate_risk_metrics(
                returns=returns_df,
                weights=weights
            )
            
            # Calculate Sharpe ratio
            sharpe_result = client.risk.calculate_sharpe_ratio(
                returns=returns_df,
                weights=weights,
                rf=0.02/252
            )
            
            # Combine results
            if 'sharpe_ratio' in sharpe_result:
                metrics['sharpe_ratio'] = sharpe_result['sharpe_ratio']
            
            risk_metrics_results[strategy_name] = metrics
            print(f"   ✅ Success! Calculated {len(metrics)} metrics")
            
        except Exception as e:
            print(f"   ❌ Failed: {e}")
            risk_metrics_results[strategy_name] = {}

print("\n📈 Risk metrics calculation complete!")

## 7. Risk Metrics Comparison

In [ ]:
# Create risk metrics comparison table
if risk_metrics_results:
    # Extract key metrics for comparison
    key_metrics = ['expected_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'var_95', 'cvar_95']
    
    comparison_data = {}
    for strategy, metrics in risk_metrics_results.items():
        strategy_metrics = {}
        for metric in key_metrics:
            strategy_metrics[metric] = metrics.get(metric, np.nan)
        comparison_data[strategy] = strategy_metrics
    
    risk_comparison_df = pd.DataFrame(comparison_data).T
    
    print("Risk Metrics Comparison:")
    print("="*80)
    print(risk_comparison_df.round(6))
    
    # Plot key metrics
    if len(risk_comparison_df) > 0:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Expected Return vs Volatility
        axes[0, 0].scatter(risk_comparison_df['volatility'], risk_comparison_df['expected_return'])
        for i, strategy in enumerate(risk_comparison_df.index):
            axes[0, 0].annotate(strategy, 
                              (risk_comparison_df.iloc[i]['volatility'], 
                               risk_comparison_df.iloc[i]['expected_return']))
        axes[0, 0].set_xlabel('Volatility')
        axes[0, 0].set_ylabel('Expected Return')
        axes[0, 0].set_title('Risk-Return Profile')
        axes[0, 0].grid(True)
        
        # Sharpe Ratio comparison
        if 'sharpe_ratio' in risk_comparison_df.columns:
            sharpe_data = risk_comparison_df['sharpe_ratio'].dropna()
            if len(sharpe_data) > 0:
                sharpe_data.plot(kind='bar', ax=axes[0, 1])
                axes[0, 1].set_title('Sharpe Ratio Comparison')
                axes[0, 1].set_ylabel('Sharpe Ratio')
                axes[0, 1].tick_params(axis='x', rotation=45)
        
        # VaR comparison
        if 'var_95' in risk_comparison_df.columns:
            var_data = risk_comparison_df['var_95'].dropna()
            if len(var_data) > 0:
                var_data.plot(kind='bar', ax=axes[1, 0], color='red', alpha=0.7)
                axes[1, 0].set_title('Value at Risk (95%)')
                axes[1, 0].set_ylabel('VaR')
                axes[1, 0].tick_params(axis='x', rotation=45)
        
        # Max Drawdown comparison
        if 'max_drawdown' in risk_comparison_df.columns:
            dd_data = risk_comparison_df['max_drawdown'].dropna()
            if len(dd_data) > 0:
                dd_data.plot(kind='bar', ax=axes[1, 1], color='orange', alpha=0.7)
                axes[1, 1].set_title('Maximum Drawdown')
                axes[1, 1].set_ylabel('Max Drawdown')
                axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
else:
    print("❌ No risk metrics to display")

## 8. Portfolio Performance Simulation

In [ ]:
# Simulate portfolio performance using historical returns
portfolio_performance = {}

for strategy_name, weights in optimization_results.items():
    if weights is not None:
        # Calculate portfolio returns
        weights_series = pd.Series(weights, index=assets)
        portfolio_returns = returns_df.dot(weights_series)
        
        # Calculate cumulative performance
        cumulative_returns = (1 + portfolio_returns).cumprod()
        
        portfolio_performance[strategy_name] = {
            'returns': portfolio_returns,
            'cumulative': cumulative_returns,
            'total_return': cumulative_returns.iloc[-1] - 1,
            'annualized_return': (cumulative_returns.iloc[-1] ** (252/len(portfolio_returns))) - 1,
            'volatility': portfolio_returns.std() * np.sqrt(252),
            'max_drawdown': (cumulative_returns / cumulative_returns.expanding().max() - 1).min()
        }

# Plot portfolio performance
if portfolio_performance:
    plt.figure(figsize=(15, 10))
    
    # Cumulative returns
    plt.subplot(2, 2, 1)
    for strategy, performance in portfolio_performance.items():
        plt.plot(performance['cumulative'], label=strategy)
    plt.title('Portfolio Cumulative Returns')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    
    # Daily returns distribution
    plt.subplot(2, 2, 2)
    returns_data = pd.DataFrame({strategy: perf['returns'] for strategy, perf in portfolio_performance.items()})
    returns_data.boxplot(ax=plt.gca())
    plt.title('Daily Returns Distribution')
    plt.xticks(rotation=45)
    
    # Risk-Return scatter
    plt.subplot(2, 2, 3)
    for strategy, performance in portfolio_performance.items():
        plt.scatter(performance['volatility'], performance['annualized_return'], 
                   s=100, label=strategy, alpha=0.7)
        plt.annotate(strategy, 
                    (performance['volatility'], performance['annualized_return']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    plt.xlabel('Volatility (Annualized)')
    plt.ylabel('Return (Annualized)')
    plt.title('Risk-Return Profile')
    plt.grid(True)
    
    # Performance summary
    plt.subplot(2, 2, 4)
    summary_data = pd.DataFrame({
        'Annualized Return': [perf['annualized_return'] for perf in portfolio_performance.values()],
        'Volatility': [perf['volatility'] for perf in portfolio_performance.values()],
        'Max Drawdown': [perf['max_drawdown'] for perf in portfolio_performance.values()]
    }, index=list(portfolio_performance.keys()))
    
    summary_data.plot(kind='bar', ax=plt.gca())
    plt.title('Performance Summary')
    plt.xticks(rotation=45)
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print summary table
    print("\nPortfolio Performance Summary:")
    print("="*80)
    print(summary_data.round(4))
else:
    print("❌ No portfolio performance data to display")

## 9. Conclusions and Next Steps

This playground demonstrates the key capabilities of the Agno BiasafeAI portfolio optimization system:

1. **Data Handling**: Generate and process return data for multiple assets
2. **Portfolio Optimization**: Run different optimization strategies (MinRisk, MaxSharpe, etc.)
3. **Risk Analysis**: Calculate comprehensive risk metrics
4. **Performance Evaluation**: Simulate and compare portfolio performance
5. **Visualization**: Create charts to understand risk-return profiles

### Next Steps:
- Try different assets and time periods
- Experiment with different risk measures (CVaR, MAD, etc.)
- Add constraints to the optimization
- Test with real market data
- Implement backtesting strategies

In [ ]:
print("🎉 Agno BiasafeAI Playground Complete!")
print("\nYou can now:")
print("1. Modify the parameters above and re-run cells")
print("2. Try different optimization strategies")
print("3. Add your own assets and data")
print("4. Experiment with constraints and custom objectives")
print("\n✨ Happy optimizing!")

## 🌐 Using the Agno Playground with Custom Assets

The enhanced Agno Playground now supports custom asset selection across all workflows!

### 🎮 Access the Playground:
- **Local**: http://localhost:7777
- **Web Interface**: https://app.agno.com/playground?endpoint=localhost%3A7777/v1

### 🛠️ Available Workflows with Custom Assets:

#### 1. **PortfolioOptimizer** 
- **Assets Field**: Enter comma-separated symbols like `"AAPL,MSFT,GOOGL,NVDA,TSLA"`
- **Examples**: 
  - Tech: `"AAPL,MSFT,GOOGL,NVDA,TSLA"`
  - Finance: `"JPM,BAC,WFC,C,GS"`
  - ETFs: `"SPY,QQQ,VTI,ARKK,XLK"`

#### 2. **RiskAnalyzer**
- **Assets Field**: Your asset universe
- **Portfolio Weights Field**: Optional weights like `"0.3,0.3,0.4"` (must sum to 1.0)
- **Example**: assets=`"AAPL,MSFT,GOOGL"`, portfolio_weights=`"0.4,0.3,0.3"`

#### 3. **DataAnalyzer** 
- **Assets Field**: Assets for market analysis
- **Analysis Type**: Choose from market_overview, correlation, trend, quality_check
- **Example**: `"XLK,XLF,XLE,XLV,XLI"` for sector analysis

#### 4. **PortfolioBacktester**
- **Assets Field**: Assets for strategy backtesting  
- **Strategy**: mean_variance, risk_parity, equal_weight, etc.
- **Example**: `"VTI,VXUS,VWO,VEA,IEMG"` for international backtesting

### 🚀 Quick Test Examples:

```
PORTFOLIO OPTIMIZATION:
Assets: "TSLA,NVDA,AMD,AAPL,MSFT"
Model: Classic
RM: MV  
OBJ: Sharpe

RISK ANALYSIS:
Assets: "JPM,BAC,WFC,C,GS"
Portfolio Weights: "0.2,0.2,0.2,0.2,0.2"
Risk Measure: VaR
Confidence Level: 0.95

DATA ANALYSIS:  
Assets: "BTC-USD,ETH-USD,ADA-USD"
Analysis Type: correlation
Time Period: 1y

BACKTESTING:
Assets: "SPY,TLT,IEF,GLD,VNQ"
Strategy: mean_variance
Backtest Period: 1y
Rebalance Frequency: monthly
```

### ✨ Pro Tips:
1. **Mix Asset Classes**: Try `"SPY,TLT,GLD,VNQ,VTEB"` for multi-asset portfolios
2. **Sector Analysis**: Use sector ETFs like `"XLK,XLF,XLE,XLV,XLI"`
3. **International**: Test with `"VTI,VXUS,VWO,VEA,IEMG"`
4. **Crypto**: Experiment with `"BTC-USD,ETH-USD,ADA-USD"`
5. **Custom Weights**: Specify exact allocations with portfolio_weights parameter